# Automatic Ticket Assignment

Importing standard libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Importing Standard NLP Libraries

In [2]:
#import Spacy
import nltk
from nltk import tokenize

# wordcloud visualisation
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

Importing Traditional Machine Learning Libraries

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# performance metics
from sklearn.metrics import confusion_matrix, classification_report, auc
from sklearn.metrics import roc_curve, accuracy_score, precision_recall_curve
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import RandomOverSampler

Using TensorFlow backend.


Importing Sequential Libraries

In [4]:
import keras.backend as K
from keras.datasets import imdb
from keras.models import Sequential, Model
from keras.layers.merge import Concatenate
from keras.layers import Input, Dropout, Flatten, Dense, Embedding, LSTM, GRU
from keras.layers import BatchNormalization, TimeDistributed, Conv1D, MaxPooling1D
from keras.constraints import max_norm, unit_norm
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint

Load the dataset

In [5]:
# Load the dataset into a Pandas dataframe called ticket and check the head of the dataset
df = pd.read_excel('Input Data Synthetic (created but not used in our project).xlsx')
print('Printing the first 5 rows of the dataset')
df.head()

Printing the first 5 rows of the dataset


,Short description,Description,Caller,Assignment group
0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0
1,outlook,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,hmjdrvpb komuaywn,GRP_0
2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,eylqgodm ybqkwiam,GRP_0
3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0


In [6]:
print('Printing the last 5 rows of the dataset')
df.tail()

Printing the last 5 rows of the dataset


,Short description,Description,Caller,Assignment group
8495,emails not coming in from zz mail,\r\n\r\nreceived from: avglmrts.vhqmtiua@gmail...,avglmrts vhqmtiua,GRP_29
8496,telephony_software issue,telephony_software issue,rbozivdq gmlhrtvp,GRP_0
8497,vip2: windows password reset for tifpdchb pedx...,vip2: windows password reset for tifpdchb pedx...,oybwdsgx oxyhwrfz,GRP_0
8498,machine nÃ£o estÃ¡ funcionando,i am unable to access the machine utilities to...,ufawcgob aowhxjky,GRP_62
8499,an mehreren pc`s lassen sich verschiedene prgr...,an mehreren pc`s lassen sich verschiedene prgr...,kqvbrspl jyzoklfx,GRP_49


In [7]:
print('The dataset contains\033[1m {}\033[0m rows and\033[1m {}\033[0m columns.'.format(df.shape[0], df.shape[1]))

The dataset contains 8500 rows and 4 columns.


In [8]:
df.isnull().sum()

Short description    8
Description          1
Caller               0
Assignment group     0
dtype: int64

We have - 
  * **8 null/missing** values present in the Short Description column. 
  * **1 null/missing** values present in the Description column.

In [9]:
df.describe().T

,count,unique,top,freq
Short description,8492,7481,password reset,38
Description,8499,7817,the,56
Caller,8500,2950,bpctwhsn kzqsbmtp,810
Assignment group,8500,74,GRP_0,3976


Following observation can be inferred from the above summary statistics of the columns:
  * The most occuring tickets in the **Short Description** column is **"Password Reset"**.
  * The most occuring tickets in the **Description** column is **"the"**.
  * The most number of calls was made by the user **"bpctwhsn kzqsbmtp"** and the customer has made a total of **810** calls.
  * A total of **3976** tickets have been assigned to **GRP_0**.

#### Analysis of column with NULL Values

In [10]:
df[pd.isnull(df).any(axis = 1)]

,Short description,Description,Caller,Assignment group
2604,NaN,\r\n\r\nreceived from: ohdrnswl.rezuibdt@gmail...,ohdrnswl rezuibdt,GRP_34
3383,NaN,\r\n-connected to the user system using teamvi...,qftpazns fxpnytmk,GRP_0
3906,NaN,-user unable tologin to vpn.\r\n-connected to...,awpcmsey ctdiuqwe,GRP_0
3910,NaN,-user unable tologin to vpn.\r\n-connected to...,rhwsmefo tvphyura,GRP_0
3915,NaN,-user unable tologin to vpn.\r\n-connected to...,hxripljo efzounig,GRP_0
3921,NaN,-user unable tologin to vpn.\r\n-connected to...,cziadygo veiosxby,GRP_0
3924,NaN,name:wvqgbdhm fwchqjor\nlanguage:\nbrowser:mic...,wvqgbdhm fwchqjor,GRP_0
4341,NaN,\r\n\r\nreceived from: eqmuniov.ehxkcbgj@gmail...,eqmuniov ehxkcbgj,GRP_0
4395,i am locked out of skype,NaN,viyglzfo ajtfzpkb,GRP_0


In [11]:
df.fillna(str(), inplace = True)
print('Replacing the NULL values with an empty string and verifying the output.')
df.isnull().sum()

Replacing the NULL values with an empty string and verifying the output.


Short description    0
Description          0
Caller               0
Assignment group     0
dtype: int64

NULL / Missing values could have treated using multiple ways, such as:
  * Replacing them with the most frequently occuring value (mode) of that column.
  * Replacing them with any default values.
  * Replacing them with an empty string.
  * Dropping the rows with NULL values. Dropping rows was avoided altogether to avoid loss of any substantial information.
  
So, we have just replaced the NULL values with an empty string.

In [12]:
df['Assignment group'].nunique()

74